<a href="https://colab.research.google.com/github/Aquib88raza/image-classification-ANKLE/blob/main/ankle_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# Project:

# Tools: Python --version
#        Scikit-learn - deep learning python library

# README:

# Dataset:

# Usefull infos:
# https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0265949

In [28]:
import os
import random
import numpy as np
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report

In [29]:
classifiers_names = [
    "KNeighborsClassifier",
    "SVC",
    'MultinomialNB',
    "DecisionTreeClassifier",
    "RandomForestClassifier"
]

classifiers = [
    KNeighborsClassifier(),
    SVC(),
    MultinomialNB(),
    DecisionTreeClassifier(),
    RandomForestClassifier()
]

parameters = [
    {'n_neighbors': [3, 5, 7, 9, 11, 13]},  # KNeighborsClassifier
    {'C': [1, 10, 50, 70, 100]}, #'gamma':['scale', 'auto'], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'class_weight': [None, 'balanced']}, # SVC
    {'alpha': [0.1, 0.5, 1.0]},  # MultinomialNB
    {'max_depth': [None, 3, 5, 7, 10]},  # DecisionTreeClassifier
    {'n_estimators': [50, 100, 200, 300, 500]}#, 'warm_start': [True, False], 'n_jobs':[None, 1, 2, 3]}  # RandomForestClassifier
]

best_models = []
acc_cross = []
var_cross = []

# Importing images


In [30]:
healthy_datset = []
ill_datset = []
dataset_images_names = []
dataset_images = []
dataset_labels = []

# PATH to files
normal_path = "/content/DATACLEANED/NORMAL"
mild_path = "/content/DATACLEANED/MILD"
ill_path = "/content/DATACLEANED/ILL"

In [31]:
# Reading - normal directory
for files in os.listdir(normal_path):
  if files.endswith(".jpg"):
    files = normal_path + '/' + files
    healthy_datset.append(files)
    dataset_labels.append("0")
  else:
    continue

#print(healthy_datset)
#print()
#print(len(healthy_datset))

In [32]:
# Reading - mid directory
for files in os.listdir(mild_path):
  if files.endswith(".jpg"):
    files = mild_path + '/' + files
    ill_datset.append(files)
    dataset_labels.append("1")
  else:
    continue

#print(ill_datset)
#print()
#print(len(ill_datset))

# Reading - ill directory
for files in os.listdir(ill_path):
  if files.endswith(".jpg"):
    files = ill_path + '/' + files
    ill_datset.append(files)
    dataset_labels.append("1")
  else:
    continue

#print(len(ill_datset))

In [33]:
dataset_images_names = healthy_datset + ill_datset

print(len(dataset_images_names))
print(len(dataset_labels))

152
152


In [34]:
#Shuffeling:
# Combine the arrays into pairs using zip
# Display the shuffled arrays
# print("Shuffled array1:", dataset_images)
# print("Shuffled array2:", dataset_labels)
combined_arrays = list(zip(dataset_images_names, dataset_labels))

# Shuffle the pairs
random.shuffle(combined_arrays)

# Unzip the shuffled pairs back into separate arrays
shuffled_array1, shuffled_array2 = zip(*combined_arrays)

# Convert the result back to lists if needed
shuffled_array1 = list(shuffled_array1)
shuffled_array2 = list(shuffled_array2)

dataset_images_names = np.array(shuffled_array1)
dataset_labels = np.array(shuffled_array2)

# Display the shuffled arrays
# print("Shuffled array1:", dataset_images)
# print("Shuffled array2:", dataset_labels)


In [35]:
#preprocesing maybe
for file in dataset_images_names: #enumerate(dataset_images_names):
  image = cv2.imread(file)

  if image is not None and not image.size == 0:
      size = (256, 256)
      image = cv2.resize(image, size, interpolation=cv2.INTER_AREA)
      #cv2.imshow('image', image)
      #cv2.waitKey(0)
      dataset_images.append(image)
  else:
      print(f"Error loading or resizing image: {file}")

In [36]:
print(len(dataset_images))

152


In [37]:
# @title Models parameters

In [38]:
for i in range(len(classifiers)):
  #for i in range(1):
  model = classifiers[i]
  model_name = classifiers_names[i]
  param_grid = parameters[i]

  # Reshape images into 2-dimensional arrays
  num_samples = len(dataset_images)
  image_shape = dataset_images[0].shape
  dataset_images_reshaped = np.reshape(dataset_images, (num_samples, -1))

  best_model = GridSearchCV(model, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=8))

  best_model.fit(dataset_images_reshaped, dataset_labels)
  best_models.append(best_model)

  params = best_model.get_params()
  print(f'{model_name} -> {params}')

  cv_accuracy = cross_val_score(best_model.best_estimator_, dataset_images_reshaped, dataset_labels, scoring='accuracy', cv=StratifiedKFold(n_splits=8))
  print(f'{model_name} - Accuracy using cross-validation: {cv_accuracy.mean()*100.0} %')
  acc_cross.append(f'{model_name} : {cv_accuracy.mean()*100.0} %')
  print(f'{model_name} - Variancy using cross-validation: {cv_accuracy.var()*100.0} %')
  var_cross.append(f'{model_name} : {cv_accuracy.var()*100.0} %')
  print()


KNeighborsClassifier -> {'cv': StratifiedKFold(n_splits=8, random_state=None, shuffle=False), 'error_score': nan, 'estimator__algorithm': 'auto', 'estimator__leaf_size': 30, 'estimator__metric': 'minkowski', 'estimator__metric_params': None, 'estimator__n_jobs': None, 'estimator__n_neighbors': 5, 'estimator__p': 2, 'estimator__weights': 'uniform', 'estimator': KNeighborsClassifier(), 'n_jobs': None, 'param_grid': {'n_neighbors': [3, 5, 7, 9, 11, 13]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': False, 'scoring': 'accuracy', 'verbose': 0}
KNeighborsClassifier - Accuracy using cross-validation: 67.10526315789474 %
KNeighborsClassifier - Variancy using cross-validation: 1.0214681440443214 %

SVC -> {'cv': StratifiedKFold(n_splits=8, random_state=None, shuffle=False), 'error_score': nan, 'estimator__C': 1.0, 'estimator__break_ties': False, 'estimator__cache_size': 200, 'estimator__class_weight': None, 'estimator__coef0': 0.0, 'estimator__decision_function_shape': 'ovr'

In [39]:
# @title Cross-validation


In [40]:
# @title Scors and Outputs


In [41]:
# Print accuracy using cross-validation
#cv_accuracy = cross_val_score(grid_search.best_estimator_, X_train, Y_train, scoring='accuracy', cv=StratifiedKFold(n_splits=8))
#print(f'{names[i]} - Accuracy using cross-validation: {cv_accuracy.mean()*100.0} %')
#acc_cross.append(f'{names[i]} : {cv_accuracy.mean()*100.0} %')
#print(f'{names[i]} - Variancy using cross-validation: {cv_accuracy.var()*100.0} %')
#var_cross.append(f'{names[i]} : {cv_accuracy.var()*100.0} %')

In [42]:
# @title Results